### Дз 2
1. Я выбрал тексты из постов группы вк [Карусель Заря](https://vk.com/club168307086). Думаю они подходят так как там довольно разнообразный лексикон и встречаются необычные словоформы.
2. Три теггера - stanza, pymorphy, natasha. Часть сравнения теггеров находится в папке [tournament](./tournament/). 
    - [main.py](./tournament/main.py) прогоняет весь турнир и выводит результаты accuracy. 
    - [tag_manual.py](./tournament/tag_manual.py) простой скриптик с UI для разметки референсного корпуса.
    - [tag_natasha.py](./tournament/tag_natasha.py), [tag_pymorphy.py](./tournament/tag_pymorphy.py), [tag_stanza.py](./tournament/tag_stanza.py) - модули, размечающие текст используя соответсвующий тэггер и сохранающие результаты в файлы в [parsed_data](./tournament/parsed_data/). 
    - [tag_toolkit.py](./tournament/tag_toolkit.py) - функции помошники для читабельности и компактности кода
3. accuracy выводится в конце исполнения [main.py](./tournament/main.py). Конвертер POS тэгов пришлось написать для pymorphy тк он использует не universal pos tags.

In [1]:
from tournament import run_tournament
run_tournament()

/home/kesha/dev/hse/nlp_hw/hw2/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:root:Ready data paths:
{'natasha': '/home/kesha/dev/hse/nlp_hw/hw2/tournament/parsed_data/natasha.xml',
 'pymorphy': '/home/kesha/dev/hse/nlp_hw/hw2/tournament/parsed_data/pymorphy.xml',
 'stanza': '/home/kesha/dev/hse/nlp_hw/hw2/tournament/parsed_data/stanza.xml'}
INFO:tournament.main:Accuracy of NATASHA: 0.82464
INFO:tournament.main:Accuracy of PYMORPHY: 0.80569
INFO:tournament.main:Accuracy of STANZA: 0.83886


4. Входные данные - 1000 положительных и 1000 отризательных отзывов. 

In [2]:
from pathlib import Path
bigrams_path = Path('./ngrams/data/bigrams.json')
trigrams_path = Path('./ngrams/data/trigrams.json')
do_parse_ngrams = not (bigrams_path.exists() and trigrams_path.exists())

Загружаем данные и парсим их, считаем частотность всех лемматизированых биграм и триграм с помощью своего модуля [parse](./ngrams/_parse.py). (Только если частотности n-грамм ещё не посчитаны)

In [3]:
from json import load, dump
from tqdm import tqdm
from collections import Counter

if do_parse_ngrams:
    from ngrams import parse_review, data_file

    pos_bigrams = Counter(); pos_trigrams = Counter()
    neg_bigrams = Counter(); neg_trigrams = Counter()

    with open(data_file, 'r', encoding='utf-8') as f:
        data = load(f)

    for review in tqdm(data['pos'][:1000]):
        bigrams, trigrams = parse_review(review)
        pos_bigrams.update(bigrams)
        pos_trigrams.update(trigrams)
    for review in tqdm(data['neg'][:1000]):
        bigrams, trigrams = parse_review(review)
        neg_bigrams.update(bigrams)
        neg_trigrams.update(trigrams)

    f"pos: {(len(pos_bigrams), len(pos_trigrams))}, neg: {(len(neg_bigrams), len(neg_trigrams))}"

2023-10-07 19:01:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-10-07 19:01:40 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |

INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |

2023-10-07 19:01:40 INFO: Using device: cpu
INFO:stanza:Using device: cpu
2023-10-07 19:01:40 INFO

Нормируем частотности и фильтруем малчастотные n-граммы

In [4]:
def normalize(count: Counter) -> dict:
    minimun_count = 5
    total = count.total()
    return { k: v/total for k, v in count.items() if v >= minimun_count }
if do_parse_ngrams:
    pos_bigrams = normalize(pos_bigrams)
    pos_trigrams = normalize(pos_trigrams)
    neg_bigrams = normalize(neg_bigrams)
    neg_trigrams = normalize(neg_trigrams)

Совмещаем биграммы с биграммами, триграммы с триграммами, а разделение на негативные и положительные указываем во внутреннем словаре.

Т.е. 
```
# Положительные отзывы
{
    'хороший + NOUN': 12,
    'не + VERB': 6
}
# Отрицательные отзывы
{
    'ужасно + VERB': 32,
    'не + VERB': 16
}
```
превратится в
```
{
    'хороший + NOUN': {
        'pos': 12,
        'neg': 0
    },
    'ужасно + VERB': {
        'pos': 0,
        'neg': 32
    },
    'не + VERB': {
        'pos': 6,
        'neg': 16
    },
}
```
Экономия памяти и удобство для последующего сравнения в наших целях

In [5]:
def merge_ngrams(pos_ngrams, neg_ngrams) -> dict:
    all_ngrams = set(pos_ngrams.keys())
    all_ngrams.update(neg_ngrams.keys())
    result = {}
    for ngram in all_ngrams:
        result[ngram] = {
            'pos': pos_ngrams[ngram] if ngram in pos_ngrams else 0,
            'neg': neg_ngrams[ngram] if ngram in neg_ngrams else 0,
        }
    return result
if do_parse_ngrams:
    bigrams = merge_ngrams(pos_bigrams, neg_bigrams)
    del pos_bigrams, neg_bigrams
    trigrams = merge_ngrams(pos_trigrams, neg_trigrams)
    del pos_trigrams, neg_trigrams
    with open(bigrams_path, 'w') as f:
        dump(bigrams, f, ensure_ascii=False, indent=4)
    with open(trigrams_path, 'w') as f:
        dump(trigrams, f, ensure_ascii=False, indent=4)
    print(f"{len(bigrams)} bigrams, {len(trigrams)} trigrams")
else:
    # Если файлы уже есть то просто их загружаем:
    with open(bigrams_path, 'r') as f:
        bigrams = load(f)
    with open(trigrams_path, 'r') as f:
        trigrams = load(f)

6389 bigrams, 10601 trigrams


Далее выявляем n-граммы, обладающие наибольшей разностью частотности между позитивными и негативными отзывами.

Задать критерий сортировки можно двумя способами: 
- разность между частотностями `get_top_diff()`. n-грамма с P('pos')=0.05,P('neg')=0.07 будет считаться БОЛЕЕ негативной чем n-грамма с P('pos')=0.01,P('neg')=0.02 т.е. в этом случае 2-1 < 7-5
- соотношениие частотностей `get_top_scale()`. n-грамма с P('pos')=0.05,P('neg')=0.07 будет считаться МЕНЕЕ негативной чем n-грамма с P('pos')=0.01,P('neg')=0.02 т.е. в этом случае 2/1 > 7/5

In [15]:
def get_top_diff(ngrams, target, other, n=5):
    def comparison_key(x):
        return x[1][target] - x[1][other]
    sorted_top = sorted(ngrams.items(), key=comparison_key, reverse=True)[:n]
    return sorted_top
def get_top_scale(ngrams, target, other, n=5):
    def comparison_key(x):
        if x[1][other] == 0:
            return 0
        else:
            return x[1][target] / x[1][other]
    sorted_top = sorted(ngrams.items(), key=comparison_key, reverse=True)[:n]
    return sorted_top

Но `get_top_diff()` хоть и кажется логичнее и первым пришло в голову, но в данной ситуации оно выдаёт не такой интересный результат:

In [16]:
print("Top 5 only positive bigrams:")
get_top_diff(bigrams, 'pos', 'neg')

Top 5 only positive bigrams:


[('T_ADJ + T_NOUN',
  {'pos': 0.016079781704781703, 'neg': 0.011468401486988848}),
 ('T_NOUN + T_NOUN',
  {'pos': 0.013831860706860707, 'neg': 0.011891263940520445}),
 ('T_ADV + T_ADJ',
  {'pos': 0.003241943866943867, 'neg': 0.0015938661710037175}),
 ('T_NOUN + T_ADJ',
  {'pos': 0.005223492723492724, 'neg': 0.003661710037174721}),
 ('T_NOUN + T_ADV',
  {'pos': 0.006860706860706861, 'neg': 0.005408921933085502})]

Наибольшая разница очевидно наблюдается у самых частотных n-грамм. А чамые частотные n-граммы в нашем случае это состоящие только из POS тэгов. Это нам мало интересного кроме того, что ADJ + NOUN имеет частотность 1.6% в положительных отзывах и только 1.1% в отрицательных.

Тогда посмотрим `get_top_scale()`:

In [17]:
print("Top 5 only positive bigrams:")
get_top_scale(bigrams, 'pos', 'neg')

Top 5 only positive bigrams:


[('T_NOUN + L_всегда',
  {'pos': 0.0006172037422037422, 'neg': 3.2527881040892195e-05}),
 ('L_всегда + T_VERB',
  {'pos': 0.0004742723492723493, 'neg': 3.2527881040892195e-05}),
 ('T_NOUN + L_спасибо',
  {'pos': 0.00033134095634095636, 'neg': 2.3234200743494423e-05}),
 ('L_хороший + L_хостинг',
  {'pos': 0.00032484407484407486, 'neg': 2.3234200743494423e-05}),
 ('L_быстро + T_CCONJ',
  {'pos': 0.00033783783783783786, 'neg': 2.7881040892193307e-05})]

`get_top_scale()` уже даёт нам n-граммы поинтереснее, с леммами. Это далеко не самые частотные n-граммы, но зато они очень характерны (в предыдущем случае) для положительных отзывов.

Посмотрим на остальные результаты применения `get_top_scale()`:

In [18]:
print("Top 5 only negative bigrams:")
get_top_scale(bigrams, 'neg', 'pos')

Top 5 only negative bigrams:


[('T_VERB + L_деньги',
  {'pos': 4.547817047817048e-05, 'neg': 0.00037639405204460967}),
 ('L_не + L_работать',
  {'pos': 7.146569646569647e-05, 'neg': 0.000525092936802974}),
 ('T_NUM + T_PROPN',
  {'pos': 4.547817047817048e-05, 'neg': 0.00029739776951672863}),
 ('T_NOUN + L_писать',
  {'pos': 4.547817047817048e-05, 'neg': 0.00029275092936802973}),
 ('T_PART + L_работать',
  {'pos': 9.095634095634096e-05, 'neg': 0.0005529739776951672})]

Тут уже появилась пара лемм "не работать"! Которая явно сигналит нам о плохом отзыве. Также видимо люди шаблон "VERB + деньги" употребляют исключительно в негативном контексте типа "потратил деньги"

In [20]:
print("Top 5 only positive trigrams:")
get_top_scale(trigrams, 'pos', 'neg')

Top 5 only positive trigrams:


[('T_NOUN + T_VERB + L_быстро',
  {'pos': 0.00016341177098340537, 'neg': 1.4204007423961214e-05}),
 ('T_NOUN + L_работать + T_ADV',
  {'pos': 0.00022677551891574623, 'neg': 2.130601113594182e-05}),
 ('L_уже + T_NUM + L_год',
  {'pos': 0.00016674670508510752, 'neg': 1.6571341994621415e-05}),
 ('L_за + T_DET + L_время',
  {'pos': 0.00010338295715276667, 'neg': 1.1836672853301011e-05}),
 ('T_PRON + L_очень + T_ADJ',
  {'pos': 0.00010004802305106452, 'neg': 1.1836672853301011e-05})]

В топе положительных триграмм у нас "NOUN + VERB + быстро". В нашем случае отзывы на хостинги так что это даже имеет смысл. "хостинг/сервер/поддержка работает/отвечает/помогает + быстро". Также видимо шаблон "уже + NUM + год" очень популярен здесь, "Я пользуюсь этим хостингом уже 6 лет", сложно представить негативный отзыв с подобным шаблоном. Тут в целом все результаты интересные но каждый я описывать и комментировать не буду. Идём дальше, к триграммам.

In [21]:
print("Top 10 only negative trigrams:")
get_top_scale(trigrams, 'neg', 'pos')

Top 10 only negative trigrams:


[('T_X + T_X + T_X',
  {'pos': 2.334453871191505e-05, 'neg': 0.00017518275822885496}),
 ('T_NOUN + L_не + L_работать',
  {'pos': 2.334453871191505e-05, 'neg': 0.00013493807052763153}),
 ('L_раз + T_ADP + T_NOUN',
  {'pos': 2.0009604610212903e-05, 'neg': 0.0001017953865383887}),
 ('T_NOUN + T_PART + L_работать',
  {'pos': 3.0014406915319353e-05, 'neg': 0.00013967273966895193}),
 ('T_PRON + L_что + T_NOUN',
  {'pos': 1.667467050851075e-05, 'neg': 7.102003711980607e-05})]

Я предположу, что на первом месте "X + X + X" (Х - значит "другое"). Я предположу, что это мат. На площадке, откуда парсились данные нецензура закрывается символами "*", которые, вероятно, stanza не считает за пунктуацию и они не отфильтровываются.